In [1]:
import os

import polars as pl

from dotenv import load_dotenv

load_dotenv(override=True)

QUERY = "SELECT * FROM DATA.DIAMONDS;"

Each of the following options will work using a Snowflake connection string. 

For more information on constructing the URI, please take a look at the 
[Connection String Examples](https://docs.snowflake.com/developer-guide/python-connector/sqlalchemy#connection-string-examples) 
on the Snowflake docs. 

## Option 1: Using Snowflake SQLAlchemy (`read_database`)

This solution currently only works with versions of Polars < 1. With version 1, it raises the following error:

`ImportError: cannot import name 'async_sessionmaker' from 'sqlalchemy.ext.asyncio'`

In [2]:
from sqlalchemy import create_engine

engine = create_engine(os.environ.get("SNOWFLAKE_URI"))
sa_con = engine.connect()


try:
    pl.read_database(QUERY, sa_con).head()
except ImportError as e:
    from pathlib import Path

    print(str(e).replace(str(Path("~").expanduser()), ""))

cannot import name 'async_sessionmaker' from 'sqlalchemy.ext.asyncio' (/Documents/GitHub/snowflake-to-polars/.venv/lib/python3.11/site-packages/sqlalchemy/ext/asyncio/__init__.py)


## Option 2: Using the Snowflake Python Connector (`read_database`)

The way the connection is being created here assumes a configuration file is present.

For more information on this, please visit
[Connecting using the connections.toml file](https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-connect#connecting-using-the-connections-toml-file)
on the Snowflake docs.

This solution (as of now) also requires that pandas is installed.

In [3]:
from snowflake.connector import connect

sf_con = connect()
pl.read_database(QUERY, sf_con).head()

CARAT,CUT,COLOR,CLARITY,DEPTH,TABLE_WIDTH,PRICE,X,Y,Z
f64,str,str,str,f64,f64,i16,f64,f64,f64
0.23,"""Ideal""","""E""","""SI2""",61.5,55.0,326,3.95,3.98,2.43
0.21,"""Premium""","""E""","""SI1""",59.8,61.0,326,3.89,3.84,2.31
0.23,"""Good""","""E""","""VS1""",56.9,65.0,327,4.05,4.07,2.31
0.29,"""Premium""","""I""","""VS2""",62.4,58.0,334,4.2,4.23,2.63
0.31,"""Good""","""J""","""SI2""",63.3,58.0,335,4.34,4.35,2.75


## Option 3: Using the Snowflake ADBC Driver (`read_database_uri`)

In [4]:
pl.read_database_uri(QUERY, os.environ.get("SNOWFLAKE_URI"), engine="adbc").head()

CARAT,CUT,COLOR,CLARITY,DEPTH,TABLE_WIDTH,PRICE,X,Y,Z
f64,str,str,str,f64,f64,"decimal[38,0]",f64,f64,f64
0.23,"""Ideal""","""E""","""SI2""",61.5,55.0,326,3.95,3.98,2.43
0.21,"""Premium""","""E""","""SI1""",59.8,61.0,326,3.89,3.84,2.31
0.23,"""Good""","""E""","""VS1""",56.9,65.0,327,4.05,4.07,2.31
0.29,"""Premium""","""I""","""VS2""",62.4,58.0,334,4.2,4.23,2.63
0.31,"""Good""","""J""","""SI2""",63.3,58.0,335,4.34,4.35,2.75
